<a href="https://colab.research.google.com/github/spatank/CIS-530/blob/master/homework_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pymagnitude

     |████████████████████████████████| 5.4MB 2.9MB/s 
  Created wheel for pymagnitude: filename=pymagnitude-0.1.120-cp36-cp36m-linux_x86_64.whl size=135918206 sha256=c8a992cd5fd0b1128bb6e25b2de67f0cbba61621593595da18b6784254db2c15
  Stored in directory: /root/.cache/pip/wheels/a2/c7/98/cb48b9db35f8d1a7827b764dc36c5515179dc116448a47c8a1
Successfully built pymagnitude


In [0]:
from pymagnitude import *
from collections import defaultdict
import gzip
from itertools import combinations
from prettytable import PrettyTable
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction import DictVectorizer
import random

In [3]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
from google.colab import drive
drive.mount('/content/drive')

# to view contents, run following line
# !ls drive/My\ Drive/CIS-530/Homework\ 5/Data

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def load_input_file(file_path):
    """
    Loads the input file to two dictionaries
    :param file_path: path to an input file
    :return: 2 dictionaries:
    1. Dictionary, where key is a target word and value is a list of paraphrases
    2. Dictionary, where key is a target word and value is a number of clusters
    """
    word_to_paraphrases_dict = {}
    word_to_k_dict = {}

    with open(file_path, 'r') as fin:
        for line in fin:
            target_word, k, paraphrases = line.split(' :: ')
            word_to_k_dict[target_word] = int(k)
            word_to_paraphrases_dict[target_word] = paraphrases.split()

    return word_to_paraphrases_dict, word_to_k_dict


def load_output_file(file_path):
    """
    :param file_path: path to an output file
    :return: A dictionary, where key is a target word and value is a list of list of paraphrases
    """
    clusterings = {}

    with open(file_path, 'r') as fin:
        for line in fin:
            target_word, _, paraphrases_in_cluster = line.strip().split(' :: ')
            paraphrases_list = paraphrases_in_cluster.strip().split()
            if target_word not in clusterings:
                clusterings[target_word] = []
            clusterings[target_word].append(paraphrases_list)

    return clusterings


def write_to_output_file(file_path, clusterings):
    """
    Writes the result of clusterings into an output file
    :param file_path: path to an output file
    :param clusterings:  A dictionary, where key is a target word and value is a list of list of paraphrases
    :return: N/A
    """
    with open(file_path, 'w') as fout:
        for target_word, clustering in clusterings.items():
            for i, cluster in enumerate(clustering):
                fout.write(f'{target_word} :: {i + 1} :: {" ".join(cluster)}\n')
        fout.close()


def get_paired_f_score(gold_clustering, predicted_clustering):
    """
    :param gold_clustering: gold list of list of paraphrases
    :param predicted_clustering: predicted list of list of paraphrases
    :return: Paired F-Score
    """
    gold_pairs = set()
    for gold_cluster in gold_clustering:
        for pair in combinations(gold_cluster, 2):
            gold_pairs.add(tuple(sorted(pair)))

    predicted_pairs = set()
    for predicted_cluster in predicted_clustering:
        for pair in combinations(predicted_cluster, 2):
            predicted_pairs.add(tuple(sorted(pair)))

    overlapping_pairs = gold_pairs & predicted_pairs

    precision = 1. if len(predicted_pairs) == 0 else float(len(overlapping_pairs)) / len(predicted_pairs)
    recall = 1. if len(gold_pairs) == 0 else float(len(overlapping_pairs)) / len(gold_pairs)
    paired_f_score = 0. if precision + recall == 0 else 2 * precision * recall / (precision + recall)

    return paired_f_score


def evaluate_clusterings(gold_clusterings, predicted_clusterings):
    """
    Displays evaluation scores between gold and predicted clusterings
    :param gold_clusterings: dictionary where key is a target word and value is a list of list of paraphrases
    :param predicted_clusterings: dictionary where key is a target word and value is a list of list of paraphrases
    :return: N/A
    """
    target_words = set(gold_clusterings.keys()) & set(predicted_clusterings.keys())

    if len(target_words) == 0:
        print('No overlapping target words in ground-truth and predicted files')
        return None

    paired_f_scores = np.zeros((len(target_words)))
    ks = np.zeros((len(target_words)))

    table = PrettyTable(['Target', 'k', 'Paired F-Score'])
    for i, target_word in enumerate(target_words):
        paired_f_score = get_paired_f_score(gold_clusterings[target_word], predicted_clusterings[target_word])
        k = len(gold_clusterings[target_word])
        paired_f_scores[i] = paired_f_score
        ks[i] = k
        table.add_row([target_word, k, f'{paired_f_score:0.4f}'])

    average_f_score = np.average(paired_f_scores, weights=ks)
    print(table)
    print(f'=> Average Paired F-Score:  {average_f_score:.4f}')

In [0]:
random.seed(123)

# TASK 3.1
def cluster_random(word_to_paraphrases_dict, word_to_k_dict):
    """
    Clusters paraphrases randomly
    :param word_to_paraphrases_dict: dictionary, where key is a target word and value is a list of paraphrases
    :param word_to_k_dict: dictionary, where key is a target word and value is a number of clusters
    :return: dictionary, where key is a target word and value is a list of list of paraphrases,
    where each list corresponds to a cluster
    """
    clusterings = {}

    for target_word in word_to_paraphrases_dict.keys():
        paraphrase_list = word_to_paraphrases_dict[target_word]
        clusters = []
        k = word_to_k_dict[target_word] # number of clusters for target word
        chosen_paraphrases = set() # keep track of any paraphrases that may not be randomly chosen
        for cluster in range(k): 
          # each word must have a cluster, each cluster must have a word
          cluster_list = random.choices(paraphrase_list, k = int(np.round(len(paraphrase_list)/k)))
          chosen_paraphrases.update(cluster_list)
          clusters.append(cluster_list)
        for paraphrase in paraphrase_list:
          if paraphrase not in chosen_paraphrases:
            # choose a random cluster list and append unassigned word to it
            random.choice(clusters).append(paraphrase) 
        clusterings[target_word] = clusters

    return clusterings

In [7]:
input_filepath = 'drive/My Drive/CIS-530/Homework 5/Data/data/dev_input.txt'
output_filepath = 'drive/My Drive/CIS-530/Homework 5/Data/data/dev_output.txt'
word_to_paraphrases_dict, word_to_k_dict = load_input_file(input_filepath)
gold_clusterings = load_output_file(output_filepath)
predicted_clusterings = cluster_random(word_to_paraphrases_dict, word_to_k_dict)
evaluate_clusterings(gold_clusterings, predicted_clusterings)

+----------------+----+----------------+
|     Target     | k  | Paired F-Score |
+----------------+----+----------------+
|    write.v     | 9  |     0.2313     |
|    party.n     | 5  |     0.2421     |
|    simple.a    | 5  |     0.1333     |
|     talk.v     | 6  |     0.3318     |
|   operate.v    | 7  |     0.2544     |
|    climb.v     | 6  |     0.2069     |
|   produce.v    | 7  |     0.2305     |
|    degree.n    | 7  |     0.2956     |
|    watch.v     | 5  |     0.2393     |
|     bank.n     | 9  |     0.2326     |
| performance.n  | 5  |     0.3240     |
|     mean.v     | 6  |     0.2431     |
|   receive.v    | 13 |     0.0603     |
|  difference.n  | 5  |     0.3724     |
|     eat.v      | 6  |     0.2922     |
|    image.n     | 9  |     0.1646     |
|     miss.v     | 8  |     0.2316     |
|     wash.v     | 13 |     0.1697     |
|    treat.v     | 8  |     0.2271     |
|   express.v    | 7  |     0.2300     |
|     note.v     | 3  |     0.5957     |
|   provide.v   

In [0]:
# word_to_paraphrases_dict, word_to_k_dict = load_input_file('drive/My Drive/CIS-530/Homework 5/Data/data/test_input.txt')
# predicted_clusterings = cluster_random(word_to_paraphrases_dict, word_to_k_dict)
# write_to_output_file('drive/My Drive/CIS-530/Homework 5/test_output_random.txt', predicted_clusterings)

In [0]:
def count_syllables(word): 

    word = word.lower() 
    # exception_add are words that need extra syllables
    # exception_del are words that need less syllables
    exception_add = ['serious','crucial']
    exception_del = ['fortunately','unfortunately'] 
    co_one = ['cool','coach','coat','coal','count','coin','coarse','coup','coif','cook','coign','coiffe','coof','court']
    co_two = ['coapt','coed','coinci']
    pre_one = ['preach']
 
    syls = 0 # added syllable number
    disc = 0 # discarded syllable number
 
    #1) if letters < 3 : return 1
    if len(word) <= 3 :
        syls = 1
        return syls
 
    #2) if doesn't end with "ted" or "tes" or "ses" or "ied" or "ies", discard "es" and "ed" at the end.
    # if it has only 1 vowel or 1 set of consecutive vowels, discard. (like "speed", "fled" etc.) 
    if word[-2:] == "es" or word[-2:] == "ed" :
        doubleAndtripple_1 = len(re.findall(r'[eaoui][eaoui]',word))
        if doubleAndtripple_1 > 1 or len(re.findall(r'[eaoui][^eaoui]',word)) > 1 :
            if word[-3:] == "ted" or word[-3:] == "tes" or word[-3:] == "ses" or word[-3:] == "ied" or word[-3:] == "ies" :
                pass
            else :
                disc += 1
 
    #3) discard trailing "e", except where ending is "le"   
    le_except = ['whole','mobile','pole','male','female','hale','pale','tale','sale','aisle','whale','while'] 
    if word[-1:] == "e" :
        if word[-2:] == "le" and word not in le_except :
            pass 
        else :
            disc += 1
 
    #4) check if consecutive vowels exists, triplets or pairs, count them as one. 
    doubleAndtripple = len(re.findall(r'[eaoui][eaoui]',word))
    tripple = len(re.findall(r'[eaoui][eaoui][eaoui]',word))
    disc += doubleAndtripple + tripple
 
    # 5) count remaining vowels in word.
    numVowels = len(re.findall(r'[eaoui]',word))
 
    # 6) add one if starts with "mc"
    if word[:2] == "mc" :
        syls+=1
 
    # 7) add one if ends with "y" but is not surrouned by vowel
    if word[-1:] == "y" and word[-2] not in "aeoui" :
        syls +=1
 
    # 8) add one if "y" is surrounded by non-vowels and is not in the last word.
    for i,j in enumerate(word) :
        if j == "y" :
            if (i != 0) and (i != len(word)-1) :
                if word[i-1] not in "aeoui" and word[i+1] not in "aeoui" :
                    syls+=1
 
    # 9) if starts with "tri-" or "bi-" and is followed by a vowel, add one. 
    if word[:3] == "tri" and word[3] in "aeoui" :
        syls+=1 
    if word[:2] == "bi" and word[2] in "aeoui" :
        syls+=1
 
    # 10) if ends with "-ian", should be counted as two syllables, except for "-tian" and "-cian"
    if word[-3:] == "ian" : 
    # and (word[-4:] != "cian" or word[-4:] != "tian") :
        if word[-4:] == "cian" or word[-4:] == "tian" :
            pass
        else :
            syls+=1
 
    # 11) if starts with "co-" and is followed by a vowel, check if exists in the double syllable dictionary, 
    #     if not, check if in single dictionary and act accordingly. 
    if word[:2] == "co" and word[2] in 'eaoui' :
        if word[:4] in co_two or word[:5] in co_two or word[:6] in co_two :
            syls+=1
        elif word[:4] in co_one or word[:5] in co_one or word[:6] in co_one :
            pass
        else :
            syls+=1
 
    # 12) if starts with "pre-" and is followed by a vowel, check if exists in the double syllable dictionary, 
    #     if not, check if in single dictionary and act accordingly. 
    if word[:3] == "pre" and word[3] in 'eaoui' :
        if word[:6] in pre_one :
            pass
        else :
            syls+=1
 
    # 13) check for "-n't" and cross match with dictionary to add syllable.
    negative = ["doesn't", "isn't", "shouldn't", "couldn't","wouldn't"] 
    if word[-3:] == "n't" :
        if word in negative :
            syls+=1
        else :
            pass  
 
    # 14) Handling the exceptional words. 
    if word in exception_del :
        disc+=1 
    if word in exception_add :
        syls+=1    
 
    # calculate the output
    return numVowels - disc + syls


def load_ngram_counts(ngram_counts_file): 
   counts = defaultdict(int) 
   with gzip.open(ngram_counts_file, 'rt') as f: 
       for line in f:
           token, count = line.strip().split('\t') 
           if token[0].islower(): 
               counts[token] = int(count) 
   return counts

In [0]:
def extract_features(paraphrase):
  features_dict = {'pos': nltk.pos_tag(paraphrase)[0][1],
                   'syl': count_syllables(paraphrase)}
  return features_dict


# TASK 3.2
def cluster_with_sparse_representation(word_to_paraphrases_dict, word_to_k_dict):
    """
    Clusters paraphrases using sparse vector representation
    :param word_to_paraphrases_dict: dictionary, where key is a target word and value is a list of paraphrases
    :param word_to_k_dict: dictionary, where key is a target word and value is a number of clusters
    :return: dictionary, where key is a target word and value is a list of list of paraphrases,
    where each list corresponds to a cluster
    """
    # Note: any vector representation should be in the same directory as this file
    vectors_root_path = 'drive/My Drive/CIS-530/Homework 5/Data/vectors/'
    vectors_path = 'coocvec-500mostfreq-window-3.filter.magnitude'
    vectors = Magnitude(vectors_root_path + vectors_path, normalized = False)

    clusterings = {}

    for target_word in word_to_paraphrases_dict.keys():
        paraphrase_list = word_to_paraphrases_dict[target_word]
        clusters = []
        k = word_to_k_dict[target_word]
        chosen_paraphrases = set() # keep track of any paraphrases that may not be chosen
        X_vector = np.zeros((len(paraphrase_list), 500))
        # X_features = []
        # build data matrix
        for idx, paraphrase in enumerate(paraphrase_list):
          if paraphrase not in vectors:
            # print(paraphrase)
            X_vector[idx, :] = random.choice(vectors)[1]
          X_vector[idx, :] = vectors.query(paraphrase)
          # features_dict = extract_features(paraphrase)
          # X_features.append(features_dict)
        vectorizer = DictVectorizer(sparse = False)

        # X_features = vectorizer.fit_transform(X_features)

        # X = np.concatenate((X_vector, X_features), axis = 1)
        X = X_vector

        # kmeans = KMeans(n_clusters = k).fit(X)
        AC = AgglomerativeClustering(n_clusters = k, affinity = 'manhattan', linkage = 'single').fit(X)

        for cluster in range(k):
          # cluster_list = [paraphrase_list[idx] 
          #                 for idx, label in enumerate(kmeans.labels_) 
          #                 if label == cluster]
          cluster_list = [paraphrase_list[idx] 
                          for idx, label in enumerate(AC.labels_) 
                          if label == cluster]
          if len(cluster_list) == 0:
            print('Empty cluster\n')
          chosen_paraphrases.update(cluster_list)
          clusters.append(cluster_list)
        for paraphrase in paraphrase_list:
          if paraphrase not in chosen_paraphrases:
            # choose a random cluster list and append unassigned word to it
            random.choice(clusters).append(paraphrase) 
        clusterings[target_word] = clusters

    return clusterings

In [11]:
input_filepath = 'drive/My Drive/CIS-530/Homework 5/Data/data/dev_input.txt'
output_filepath = 'drive/My Drive/CIS-530/Homework 5/Data/data/dev_output.txt'
word_to_paraphrases_dict, word_to_k_dict = load_input_file(input_filepath)
gold_clusterings = load_output_file(output_filepath)
predicted_clusterings = cluster_with_sparse_representation(word_to_paraphrases_dict, word_to_k_dict)
evaluate_clusterings(gold_clusterings, predicted_clusterings)

+----------------+----+----------------+
|     Target     | k  | Paired F-Score |
+----------------+----+----------------+
|    write.v     | 9  |     0.3669     |
|    party.n     | 5  |     0.3882     |
|    simple.a    | 5  |     0.3590     |
|     talk.v     | 6  |     0.6338     |
|   operate.v    | 7  |     0.2844     |
|    climb.v     | 6  |     0.3375     |
|   produce.v    | 7  |     0.4698     |
|    degree.n    | 7  |     0.4182     |
|    watch.v     | 5  |     0.3300     |
|     bank.n     | 9  |     0.3000     |
| performance.n  | 5  |     0.4840     |
|     mean.v     | 6  |     0.3714     |
|   receive.v    | 13 |     0.1357     |
|  difference.n  | 5  |     0.5337     |
|     eat.v      | 6  |     0.4341     |
|    image.n     | 9  |     0.3332     |
|     miss.v     | 8  |     0.3333     |
|     wash.v     | 13 |     0.3602     |
|    treat.v     | 8  |     0.4077     |
|   express.v    | 7  |     0.4187     |
|     note.v     | 3  |     0.6400     |
|   provide.v   

In [0]:
# word_to_paraphrases_dict, word_to_k_dict = load_input_file('drive/My Drive/CIS-530/Homework 5/Data/data/test_input.txt')
# predicted_clusterings = cluster_with_sparse_representation(word_to_paraphrases_dict, word_to_k_dict)
# write_to_output_file('drive/My Drive/CIS-530/Homework 5/test_output_sparse.txt', predicted_clusterings)

In [0]:
# TASK 3.3
def cluster_with_dense_representation(word_to_paraphrases_dict, word_to_k_dict):
    """
    Clusters paraphrases using sparse vector representation
    :param word_to_paraphrases_dict: dictionary, where key is a target word and value is a list of paraphrases
    :param word_to_k_dict: dictionary, where key is a target word and value is a number of clusters
    :return: dictionary, where key is a target word and value is a list of list of paraphrases,
    where each list corresponds to a cluster
    """
    # Note: any vector representation should be in the same directory as this file
    vectors_root_path = 'drive/My Drive/CIS-530/Homework 5/Data/vectors/'
    vectors_path = 'GoogleNews-vectors-negative300.filter.magnitude'
    vectors = Magnitude(vectors_root_path + vectors_path, normalized = True)

    clusterings = {}

    for target_word in word_to_paraphrases_dict.keys():
        paraphrase_list = word_to_paraphrases_dict[target_word]
        clusters = []
        k = word_to_k_dict[target_word]
        chosen_paraphrases = set() # keep track of any paraphrases that may not be chosen
        X_vector = np.zeros((len(paraphrase_list), 300))
        # X_features = []
        # build data matrix
        for idx, paraphrase in enumerate(paraphrase_list):
          # if paraphrase not in vectors:
          #   # print(paraphrase)
          #   X_vector[idx, :] = random.choice(vectors)[1]
          X_vector[idx, :] = vectors.query(paraphrase)
          # features_dict = extract_features(paraphrase)
          # X_features.append(features_dict)
        vectorizer = DictVectorizer(sparse = False)

        # X_features = vectorizer.fit_transform(X_features)

        # X = np.concatenate((X_vector, X_features), axis = 1)
        X = X_vector

        # kmeans = KMeans(n_clusters = k).fit(X)
        AC = AgglomerativeClustering(n_clusters = k, affinity = 'manhattan', linkage = 'single').fit(X)

        for cluster in range(k):
          # cluster_list = [paraphrase_list[idx] 
          #                 for idx, label in enumerate(kmeans.labels_) 
          #                 if label == cluster]
          cluster_list = [paraphrase_list[idx] 
                          for idx, label in enumerate(AC.labels_) 
                          if label == cluster]
          if len(cluster_list) == 0:
            print('Empty cluster\n')
          chosen_paraphrases.update(cluster_list)
          clusters.append(cluster_list)
        for paraphrase in paraphrase_list:
          if paraphrase not in chosen_paraphrases:
            # choose a random cluster list and append unassigned word to it
            random.choice(clusters).append(paraphrase) 
        clusterings[target_word] = clusters

    return clusterings

In [13]:
input_filepath = 'drive/My Drive/CIS-530/Homework 5/Data/data/dev_input.txt'
output_filepath = 'drive/My Drive/CIS-530/Homework 5/Data/data/dev_output.txt'
word_to_paraphrases_dict, word_to_k_dict = load_input_file(input_filepath)
gold_clusterings = load_output_file(output_filepath)
predicted_clusterings = cluster_with_dense_representation(word_to_paraphrases_dict, word_to_k_dict)
evaluate_clusterings(gold_clusterings, predicted_clusterings)

+----------------+----+----------------+
|     Target     | k  | Paired F-Score |
+----------------+----+----------------+
|    write.v     | 9  |     0.3400     |
|    party.n     | 5  |     0.4323     |
|    simple.a    | 5  |     0.3448     |
|     talk.v     | 6  |     0.6504     |
|   operate.v    | 7  |     0.3124     |
|    climb.v     | 6  |     0.3176     |
|   produce.v    | 7  |     0.4548     |
|    degree.n    | 7  |     0.4315     |
|    watch.v     | 5  |     0.5300     |
|     bank.n     | 9  |     0.6154     |
| performance.n  | 5  |     0.4718     |
|     mean.v     | 6  |     0.3646     |
|   receive.v    | 13 |     0.2141     |
|  difference.n  | 5  |     0.5107     |
|     eat.v      | 6  |     0.4614     |
|    image.n     | 9  |     0.3278     |
|     miss.v     | 8  |     0.3964     |
|     wash.v     | 13 |     0.3403     |
|    treat.v     | 8  |     0.3903     |
|   express.v    | 7  |     0.4206     |
|     note.v     | 3  |     0.8400     |
|   provide.v   

In [0]:
# word_to_paraphrases_dict, word_to_k_dict = load_input_file('drive/My Drive/CIS-530/Homework 5/Data/data/test_input.txt')
# predicted_clusterings = cluster_with_dense_representation(word_to_paraphrases_dict, word_to_k_dict)
# write_to_output_file('drive/My Drive/CIS-530/Homework 5/test_output_dense.txt', predicted_clusterings)

In [0]:
# TASK 3.4

def choose_k(paraphrase_list, vectors):

  silhouette_scores = []
  for k in range(2, min(20, len(paraphrase_list))):
    X_vector = np.zeros((len(paraphrase_list), 300))
    for idx, paraphrase in enumerate(paraphrase_list):
      X_vector[idx, :] = vectors.query(paraphrase)
    X = X_vector
    AC = AgglomerativeClustering(n_clusters = k, affinity = 'manhattan', linkage = 'single').fit(X)
    cluster_labels = AC.fit_predict(X)
    silhouette_average = silhouette_score(X, cluster_labels, metric = 'manhattan')
    silhouette_scores.append(silhouette_average)
  if len(silhouette_scores) == 0:
    # k_choice = random.randint(1,2) # randomly choose between either 1 or 2 clusters
    k_choice = 1
  else:
    k_choice = range(2, min(20, len(paraphrase_list)))[np.argmax(silhouette_scores)]
  return k_choice

def cluster_with_no_k(word_to_paraphrases_dict):
    """
    Clusters paraphrases using any vector representation
    :param word_to_paraphrases_dict: dictionary, where key is a target word and value is a list of paraphrases
    :return: dictionary, where key is a target word and value is a list of list of paraphrases,
    where each list corresponds to a cluster
    """
    # Note: any vector representation should be in the same directory as this file
    vectors_root_path = 'drive/My Drive/CIS-530/Homework 5/Data/vectors/'
    vectors_path = 'GoogleNews-vectors-negative300.magnitude'
    # vectors_path = 'glove.840B.300d.magnitude'
    vectors = Magnitude(vectors_root_path + vectors_path, normalized = True)

    clusterings = {}

    for target_word in word_to_paraphrases_dict.keys():
        paraphrase_list = word_to_paraphrases_dict[target_word]
        clusters = []
        k = choose_k(paraphrase_list, vectors)
        chosen_paraphrases = set() # keep track of any paraphrases that may not be chosen
        X_vector = np.zeros((len(paraphrase_list), 300))
        for idx, paraphrase in enumerate(paraphrase_list):
          X_vector[idx, :] = vectors.query(paraphrase)

        vectorizer = DictVectorizer(sparse = False)

        X = X_vector

        AC = AgglomerativeClustering(n_clusters = k, affinity = 'manhattan', linkage = 'single').fit(X)

        for cluster in range(k):
          cluster_list = [paraphrase_list[idx] 
                          for idx, label in enumerate(AC.labels_) 
                          if label == cluster]
          if len(cluster_list) == 0:
            print('Empty cluster.\n')
          chosen_paraphrases.update(cluster_list)
          clusters.append(cluster_list)
        for paraphrase in paraphrase_list:
          if paraphrase not in chosen_paraphrases:
            # choose a random cluster list and append unassigned word to it
            random.choice(clusters).append(paraphrase) 
        clusterings[target_word] = clusters

    return clusterings

In [16]:
input_filepath = 'drive/My Drive/CIS-530/Homework 5/Data/data/dev_input.txt'
output_filepath = 'drive/My Drive/CIS-530/Homework 5/Data/data/dev_output.txt'
word_to_paraphrases_dict, word_to_k_dict = load_input_file(input_filepath)
gold_clusterings = load_output_file(output_filepath)
predicted_clusterings = cluster_with_no_k(word_to_paraphrases_dict)
evaluate_clusterings(gold_clusterings, predicted_clusterings)

DatabaseError: ignored

In [0]:
# word_to_paraphrases_dict, _ = load_input_file('drive/My Drive/CIS-530/Homework 5/Data/data/test_nok_input.txt')
# predicted_clusterings = cluster_with_no_k(word_to_paraphrases_dict)
# write_to_output_file('drive/My Drive/CIS-530/Homework 5/test_output_nok.txt', predicted_clusterings)